# Wrapping a subset of a very large library

Sometimes, for a very large library, only a subset of available *C++* components is useful for end-users.
Wrapping such libraries therefore requires **AutoWIG** to be able to consider only a subset of the *C++* components during the `Generate` step.
The **Clang** library is a complete *C*/*C++* compiler.
**Clang** is a great tool, but its stable *Python* interface (i.e. **libclang**) is lacking some useful features that are needed by **AutoWIG**.
In particular, class template specializations are not available in the abstract syntax tree.
Fortunately, most of the classes that would be needed during the traversal of the *C++* abstract syntax tree are not template specializations.
We therefore proposed to bootstrap the **Clang** *Python* bindings using the `libclang` `parser` of **AutoWIG**.
This new **Clang** *Python* interface is called **PyClangLite** and is able to parse class template specializations.
As for **libclang**, this interface is proposed only for a subset of the **Clang** library sufficient enough for proposing the new `pyclanglite` `parser`.


First, we need:

* to detect if the operating system (OS) is a Windows OS or a Unix OS.

In [1]:
import platform
is_windows = any(platform.win32_ver())

* to detect the version of *Python* installed and save it in the `PYTHON_VERSION` environment variable.

In [2]:
import os
import sys
os.environ['PYTHON_VERSION'] = str(sys.version_info.major) + '.' + str(sys.version_info.minor)

* to import **AutoWIG**.

In [3]:
import autowig

Then, in addition to the **Clang** libraries, the **ClangLite** library is needed in order to have access to some functionalities.
To do so, we use available **Conda** recipes.

In [4]:
if is_windows:
    !conda build --python=%PYTHON_VERSION% ..\git\ClangLite\bin\conda\libclanglite -c statiskit -c conda-forge
else:
    !conda build --python=$PYTHON_VERSION ../git/ClangLite/bin/conda/libclanglite -c statiskit -c conda-forge
!conda install -y libclanglite --use-local -c statiskit -c conda-forge

BUILD START: libclanglite-3.8.1-py27_0

The following NEW packages will be INSTALLED:

    ca-certificates: 2017.7.27.1-0      conda-forge
    certifi:         2017.7.27.1-py27_0 conda-forge
    clang:           3.8.1-0            statiskit  
    coverage:        4.4.1-py27_0       conda-forge
    icu:             58.1-1             conda-forge
    libboost:        1.61.0-py27_0      statiskit  
    libdev:          1.0.0-py27_0       statiskit  
    librun:          1.0.0-0            statiskit  
    llvm:            3.8.1-0            statiskit  
    ncurses:         5.9-10             conda-forge
    nose:            1.3.7-py27_2       conda-forge
    openssl:         1.0.2l-0           conda-forge
    path.py:         10.3.1-py27_0      conda-forge
    pip:             9.0.1-py27_0       conda-forge
    python:          2.7.13-1           conda-forge
    python-dev:      1.0.0-py27_0       statiskit  
    python-scons:    3.0.0-py27_0       statiskit  
    pyyaml:          3.12-py2

scons: done building targets.

set +ev
+ set +ev
INFO:conda_build.build:Packaging libclanglite-3.8.1-py27_0
fatal: Not a git repository: /home/main/miniconda/conda-bld/libclanglite_1505394709024/work/../../../.git/modules/share/git/ClangLite
fatal: Not a git repository: /home/main/miniconda/conda-bld/libclanglite_1505394709024/work/../../../.git/modules/share/git/ClangLite
number of files: 2
Fixing permissions
patchelf: file: /home/main/miniconda/conda-bld/libclanglite_1505394709024/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_/lib/libclanglite.so
    setting rpath to: $ORIGIN/.
Fixing permissions
Detected hard-coded path in binary file lib/libclanglite.so
/home/main/miniconda/conda-bld/linux-64/libclanglite-3.8.1-py27_0.tar.bz2
updating index in: /home/main/miniconda/conda-bld
updating index in: /home/main/miniconda/conda-bld/noarch
up

Once these preliminaries are done, we can proceed to the actual generation of wrappers for the **Clang** library.
For this, we create an empty Abstract Semantic Graph (ASG).

In [5]:
asg = autowig.AbstractSemanticGraph()

We then parse the `tool.h` header of the **ClangLite** library with relevant compilation flags.

In [6]:
%%time

try:
    from path import path as Path
except:
    from path import Path
prefix = Path(sys.prefix).abspath()
if is_windows:
    headers = [prefix/'Library'/'include'/'clanglite'/'tool.h']
else:
    headers = [prefix/'include'/'clanglite'/'tool.h']
    
import six
if six.PY2:
    autowig.parser.plugin = 'libclang'
    
flags = ['-x', 'c++', '-std=c++11',
         '-D__STDC_LIMIT_MACROS',
         '-D__STDC_CONSTANT_MACROS']
if is_windows:
    flags.append('-I' + str((prefix/'Library'/'include').abspath()))
else:
    flags.append('-I' + str((prefix/'include').abspath()))
    
asg = autowig.parser(asg,
                     headers,
                     flags = flags,
                     bootstrap = False,
                     silent = True)

CPU times: user 2min 37s, sys: 396 ms, total: 2min 38s
Wall time: 2min 38s


Since most of **AutoWIG** guidelines are respected in the **Clang** library, the `default` `controller` implementation could be suitable.
Nevertheless, we need to force some *C++* components to be wrapped or not.
We therefore implements a new `controller`.

In [7]:
def clanglite_controller(asg):
    
    for node in asg['::boost::python'].classes(nested = True):
        node.is_copyable = True
        
    for node in asg.classes():
        node.boost_python_export = False
    for node in asg.functions(free=True):
        node.boost_python_export = False
    for node in asg.variables(free = True):
        node.boost_python_export = False
    for node in asg.enumerations():
        node.boost_python_export = False
    for node in asg.enumerators():
        if node.parent.boost_python_export:
            node.boost_python_export = False
    for node in asg.typedefs():
        node.boost_python_export = False
            
    from autowig.default_controller import refactoring
    asg = refactoring(asg)

    if autowig.parser.plugin == 'libclang':
        for fct in asg.functions(free=False):
            asg._nodes[fct._node]['_is_virtual'] = False
            asg._nodes[fct._node]['_is_pure'] = False
        asg['class ::clang::QualType'].is_abstract = False
        asg['class ::clang::QualType'].is_copyable = True
        asg['class ::llvm::StringRef'].is_abstract = False
        asg['class ::llvm::StringRef'].is_copyable = True
        asg['class ::clang::FileID'].is_abstract = False
        asg['class ::clang::FileID'].is_copyable = True
        asg['class ::clang::SourceLocation'].is_abstract = False
        asg['class ::clang::SourceLocation'].is_copyable = True
        asg['class ::clang::TemplateArgument'].is_abstract = False
        asg['class ::clang::TemplateArgument'].is_copyable = True
        for cls in ['::clang::FriendDecl', '::clang::CapturedDecl', '::clang::OMPThreadPrivateDecl',
                    '::clang::NonTypeTemplateParmDecl', '::clang::TemplateArgumentList', '::clang::ImportDecl',
                    '::clang::TemplateTemplateParmDecl', '::clang::CapturedDecl', '::clang::OMPThreadPrivateDecl',
                    '::clang::NonTypeTemplateParmDecl', '::clang::TemplateArgumentList', '::clang::ImportDecl',
                    '::clang::TemplateTemplateParmDecl']:
            asg['class ' + cls].is_abstract = False
        
    asg['class ::boost::python::api::object'].boost_python_export = True
    asg['class ::boost::python::list'].boost_python_export = True 
    asg['class ::boost::python::str'].boost_python_export = True 
        
    subset = []
    classes = [asg['class ::clang::QualType'],
               asg['class ::clang::Type'],
               asg['class ::clang::Decl']]
    subset += classes
    for cls in classes:
        subset += cls.subclasses(recursive=True)
    for cls in subset:
        if not cls.globalname.strip('class ') in ['::clang::QualType',
                                                  '::llvm::StringRef',
                                                  '::clang::FileID',
                                                  '::clang::SourceLocation',
                                                  '::clang::TemplateArgument',
                                                  '::clang::FriendDecl',
                                                  '::clang::CapturedDecl',
                                                  '::clang::OMPThreadPrivateDecl',
                                                  '::clang::NonTypeTemplateParmDecl',
                                                  '::clang::TemplateArgumentList',
                                                  '::clang::ImportDecl',
                                                  '::clang::TemplateTemplateParmDecl']:
            cls.is_copyable = False
        else:
            cls.is_copyable = True
    subset.append(asg['class ::llvm::StringRef'])

    subset.append(asg['class ::clang::ASTUnit'])
    subset.append(asg['class ::clang::ASTContext'])
    subset.append(asg['class ::clang::SourceManager'])
    subset.append(asg['class ::clang::FileID'])

    subset.append(asg['class ::clang::SourceLocation'])

    subset.append(asg['class ::clang::CXXBaseSpecifier'])
    subset.append(asg['class ::clang::DeclContext'])
    subset.append(asg['class ::clang::TemplateArgument'])

    subset.append(asg['class ::clang::TemplateArgumentList'])
    subset.append(asg['enum ::clang::Type::TypeClass'])
    subset.append(asg['enum ::clang::AccessSpecifier'])
    subset.append(asg['enum ::clang::LinkageSpecDecl::LanguageIDs'])
    subset.append(asg['enum ::clang::BuiltinType::Kind'])
    subset.append(asg['enum ::clang::TemplateArgument::ArgKind'])
    subset.append(asg['enum ::clang::Decl::Kind'])
    subset.extend(asg.nodes('::clanglite::build_ast_from_code_with_args'))

    for node in subset:
        node.boost_python_export = True
        
    for fct in asg['::clanglite'].functions():
        if not fct.localname == 'build_ast_from_code_with_args':
            fct.parent = fct.parameters[0].qualified_type.desugared_type.unqualified_type
        fct.boost_python_export = True
        
    for mtd in asg['class ::clang::ASTContext'].methods(pattern='.*getSourceManager.*'):
        if mtd.return_type.globalname == 'class ::clang::SourceManager &':
                mtd.boost_python_export = True
                break
                
    if autowig.parser.plugin == 'libclang':
        for node in (asg.functions(pattern='.*(llvm|clang).*_(begin|end)')
                     + asg.functions(pattern='::clang::CXXRecordDecl::getCaptureFields')
                     + asg.functions(pattern='.*(llvm|clang).*getNameAsString')
                     + asg.nodes('::clang::NamedDecl::getQualifiedNameAsString')
                     + asg.functions(pattern='.*::clang::ObjCProtocolDecl')
                     + asg.nodes('::clang::ObjCProtocolDecl::collectInheritedProtocolProperties')
                     + asg.nodes('::clang::ASTUnit::LoadFromASTFile')
                     + asg.nodes('::clang::ASTUnit::getCachedCompletionTypes')
                     + asg.nodes('::clang::ASTUnit::getBufferForFile')
                     + asg.nodes('::clang::CXXRecordDecl::getCaptureFields')
                     + asg.nodes('::clang::ASTContext::SectionInfos')
                     + asg.nodes('::clang::ASTContext::getAllocator')
                     + asg.nodes('::clang::ASTContext::getObjCEncoding.*')
                     + asg.nodes('::clang::ASTContext::getAllocator')
                     + asg.nodes('::clang::QualType::getAsString')
                     + asg.nodes('::clang::SourceLocation::printToString')
                     + asg['class ::llvm::StringRef'].methods()):
            node.boost_python_export = False
            
    if autowig.parser.plugin == 'clanglite':
        for mtd in asg['class ::clang::Decl'].methods():
            if mtd.localname == 'hasAttr':
                mtd.boost_python_export = False
                
    import sys
    try:
        from path import path as Path
    except:
        from path import Path
        
    for header in (Path(sys.prefix)/'include'/'clang').walkfiles('*.h'):
        asg[header.abspath()].is_external_dependency = False
    
    return asg

This `controller` is then dynamically registered and used on the ASG.

In [8]:
%%time
autowig.controller['clanglite'] = clanglite_controller
autowig.controller.plugin = 'clanglite'
asg = autowig.controller(asg)

CPU times: user 16.6 s, sys: 32 ms, total: 16.6 s
Wall time: 16.7 s


In order to wrap a subset of the **Clang** library, we need to select the `boost_python_internal` `generator` implementation.

In [9]:
%%time
autowig.generator.plugin = 'boost_python_pattern'
wrappers = autowig.generator(asg,
                             module = Path('.')/'..'/'git'/'ClangLite'/'src'/'py'/'_clanglite.cpp',
                             decorator = Path('.')/'..'/'git'/'ClangLite'/'src'/'py'/'basic'/'_clanglite.py',
                             closure = False)

CPU times: user 8.63 s, sys: 24 ms, total: 8.65 s
Wall time: 8.62 s


The wrappers are only generated in-memory.
It is therefore needed to write them on the disk to complete the process.

In [10]:
%%time
wrappers.write()

CPU times: user 1min 59s, sys: 48 ms, total: 1min 59s
Wall time: 1min 59s


Here is the list of the generated wrappers (untracked files).

In [11]:
!cd ../git/ClangLite && git status

HEAD detached at 63dec09
Untracked files:
  (use "git add <file>..." to include in what will be committed)

	src/py/_clanglite.cpp
	src/py/_clanglite.h
	src/py/basic/
	src/py/clanglite/_clanglite.py
	src/py/wrapper_0418dea67efc501bbbeb15890bd22f08.cpp
	src/py/wrapper_055e36c244395ee3a248c7839b4485ca.cpp
	src/py/wrapper_0a741a99c7fd512dbe7c31ae23782c5f.cpp
	src/py/wrapper_0c11b148868f50d9aba81b8d12b3ed37.cpp
	src/py/wrapper_0dd00d729e905a24a952147ef6cb1f26.cpp
	src/py/wrapper_134a185b7d1855029390fd1393cbf65e.cpp
	src/py/wrapper_13b544d609775d6c82f349029c56dab2.cpp
	src/py/wrapper_14c4bbdd8f0a57b0a17277a678e9b9df.cpp
	src/py/wrapper_170cacd8da615406967b5b7d712829ee.cpp
	src/py/wrapper_17505392838a5ddf8a16f3bdbb8f586b.cpp
	src/py/wrapper_179220ac78145becbf9b283ff6879c57.cpp
	src/py/wrapper_185117225e1a5743866dee867a4d2c40.cpp
	src/py/wrapper_19b8049789335462bac78de513b121a7.cpp
	src/py/wrapper_19f2e9be2a19546ea2b5c16dcb943e36.cpp
	src/py/wrapper_1ab4808ba4c9580baf99795b73ebb3b3.cpp
	src/p

And here, we present the wrappers for the `clang::Decl` class.

In [12]:
if is_windows:
    !pygmentize ..\git\ClangLite\src\py\wrapper_a6aedb4654a55a40aeecf4b1dc5fcc98.cpp
else:
    !pygmentize ../git/ClangLite/src/py/wrapper_a6aedb4654a55a40aeecf4b1dc5fcc98.cpp

#include "_clanglite.h"



namespace autowig
{
    class Wrap_a6aedb4654a55a40aeecf4b1dc5fcc98 : public ::clang::Decl, public boost::python::wrapper< class ::clang::Decl >
    {
        public:
            

        protected:
            

        private:
            

    };

}

#if defined(_MSC_VER)
    #if (_MSC_VER == 1900)
namespace boost
{
    template <> autowig::Wrap_a6aedb4654a55a40aeecf4b1dc5fcc98 const volatile * get_pointer<autowig::Wrap_a6aedb4654a55a40aeecf4b1dc5fcc98 const volatile >(autowig::Wrap_a6aedb4654a55a40aeecf4b1dc5fcc98 const volatile *c) { return c; }
    template <> class ::clang::Decl const volatile * get_pointer<class ::clang::Decl const volatile >(class ::clang::Decl const volatile *c) { return c; }
}
    #endif
#endif



void wrapper_a6aedb4654a55a40aeecf4b1dc5fcc98()
{

    std::string name_7bbff48d109853e88270b3595c663a99 = boost::python::extract< std::string >(boost::python::scope().attr("__name__") + ".clang");
    boost::python::object module_7bbff

    class_a6aedb4654a55a40aeecf4b1dc5fcc98.def("get_non_closure_context", method_pointer_365b951bc2d15b808b190fd04736e0a0, boost::python::return_value_policy< boost::python::reference_existing_object >(), "");
    class_a6aedb4654a55a40aeecf4b1dc5fcc98.def("get_owning_module_id", method_pointer_c3a47500a04b5b5c98898dbf1a0808b0, "");
    class_a6aedb4654a55a40aeecf4b1dc5fcc98.def("get_parent_function_or_method", method_pointer_c274d9fc82b054a2917b5a20a58ab0c4, boost::python::return_value_policy< boost::python::reference_existing_object >(), "");
    class_a6aedb4654a55a40aeecf4b1dc5fcc98.def("get_parent_function_or_method", method_pointer_246fbc01241954d7a01ffb0597f6672a, boost::python::return_value_policy< boost::python::reference_existing_object >(), "");
    class_a6aedb4654a55a40aeecf4b1dc5fcc98.def("get_previous_decl", method_pointer_f2c5176936bb571b80f452a19f20bc4f, boost::python::return_value_policy< boost::python::reference_existing_object >(), "");
    class_a6aedb4654a55a40aee

Once the wrappers are written on the disk, the bingings must be compiled and installed.
This can be done using the **SCons** `py`  target.

In [13]:
if is_windows:
    !conda build --python=%PYTHON_VERSION% ..\git\ClangLite\bin\conda\python-clanglite -c statiskit -c conda-forge
else:
    !conda build --python=$PYTHON_VERSION ..\git\ClangLite\bin\conda\python-clanglite -c statiskit -c conda-forge
!conda install -y python-clanglite --use-local -c statiskit -c conda-forge --force

Fetching package metadata ...............
Solving package specifications: .

Package plan for installation in environment /home/main/miniconda:

The following NEW packages will be INSTALLED:

    python-clanglite: 3.8.1-py27_0 local



Here is a report concerning objects wrapped using this notebook.

In [14]:
import fp17
fp17.report(asg)

Headers: 687 (292907 SLOC)
Enumerations: 602 (1.0%)
Variables: 146 (0.0%)
Fields: 2342 (0.77%)
Functions: 2815 (0.04%)
Methods: 17601 (11.6%)
Classes: 2202 (6.4%)
